# ANALYSING SALES USING PANDAS PROJECT: CSV FILE(S)

## Author(s): Nfayem Imoro

The program performs data analysis, using pandas, on data acquired from a csv file to answer the following questions and provide insights to improve business decisions:
1. What is the total sales for each month?
2. What is the average daily sales for each month?
3. What region generated the highest sales?
4. Which product category had the lowest impact?
5. Identify the top-selling products
6. Based on your analysis what recommendation would you give to improve sales in November?

### Sales DataFrame

In [8]:
import pandas as pd

s_df = pd.read_csv("Sales-230913-134411.csv")
s_df

,Date,Region,Category,Sales($),ProductID
0,01/09/2023,East,Electronics,1200.50,E1234
1,02/09/2023,West,Apparel,350.75,A4567
2,03/09/2023,North,Furniture,890.20,F7890
3,04/09/2023,South,Electronics,550.40,E2345
4,05/09/2023,East,Appliances,420.60,A6789
...,...,...,...,...,...
56,27/10/2023,North,Furniture,870.20,F3457
57,28/10/2023,South,Electronics,730.40,E4568
58,29/10/2023,East,Appliances,410.60,A6789
59,30/10/2023,West,Apparel,320.25,A7891


#### 1. What is the total sales for each month?

In [9]:
import calendar

# Convert the 'Date' column to DateTime format
s_df["Date"] = pd.to_datetime(s_df["Date"], format="%d/%m/%Y")

# Extract the month from the 'Date' column
s_df["Month"] = s_df["Date"].dt.month

# Group by month and calculate total sales for each month
total_sales_per_month = s_df.groupby("Month")["Sales($)"].sum()

# Convert numeric month to text equivalent
total_sales_per_month.index = total_sales_per_month.index.map(lambda x: calendar.month_name[x])

# Format sales totals with commas and keep them rounded to 2 decimal places
total_sales_per_month = total_sales_per_month.map(lambda x: "$ {:,.2f}".format(x))

# Print output indicating total sales for each month
print("Total sales for each month:")
print(total_sales_per_month)

Total sales for each month:
Month
September    $ 17,924.40
October      $ 17,354.65
Name: Sales($), dtype: object


#### 2. What is the average daily sales for each month?

In [10]:
# Group by month and calculate average sales for each month
avg_sales_per_month = s_df.groupby("Month")["Sales($)"].mean()

# Convert numeric month to text equivalent
avg_sales_per_month.index = avg_sales_per_month.index.map(lambda x: calendar.month_name[x])

# Format sales averages with commas and keep them rounded to 2 decimal places
avg_sales_per_month = avg_sales_per_month.map(lambda x: "$ {:,.2f}".format(x))

# Print output indicating average sales for each month
print("Average sales for each month:")
print(avg_sales_per_month)

Average sales for each month:
Month
September    $ 597.48
October      $ 559.83
Name: Sales($), dtype: object


#### 3. What region generated the highest sales?

In [11]:
# Group sales data by region and calculate total sales for each region
sales_per_region = s_df.groupby("Region")["Sales($)"].sum()

# Identify the region with the highest total sales
region_highest_sales = sales_per_region.idxmax()

# Format sales totals with commas and keep them rounded to 2 decimal places
sales_per_region = sales_per_region.map(lambda x: "{:,.2f}".format(x))

# Print both the region with the highest sales and its corresponding total sales
print(f"Region with highest sales: {region_highest_sales}")
print(f"Total sales for this region: $ {sales_per_region[region_highest_sales]}")

Region with highest sales: North
Total sales for this region: $ 12,767.00


#### 4. Which product category had the lowest impact?

In [12]:
# Count the unique quantity for each product per daily purchase and add as a column
s_df["Quantity"] = s_df.groupby("ProductID")["ProductID"].transform('nunique')


# Group quantity data by category and calculate total quantity for each category
quantity_per_category = s_df.groupby("Category")["Quantity"].sum()

# Identify the category with the lowest total quantity
category_lowest_quantity = quantity_per_category.idxmin()


# Group sales data by category and calculate total sales for each category
sales_per_category = s_df.groupby("Category")["Sales($)"].sum()

# Identify the category with the lowest total sales
category_lowest_sales = sales_per_category.idxmin()

# Format sales totals with commas and keep them rounded to 2 decimal places
sales_per_category = sales_per_category.map(lambda x: "$ {:,.2f}".format(x))


# Print both the category with the lowest quantity and its corresponding total quantity
print(f"Low impact category based on quantity: {category_lowest_quantity}")
print(f"Total quantity for this category: {quantity_per_category[category_lowest_quantity]}")
print()

# Print both the category with the lowest sales and its corresponding total sales
print(f"Low impact category based on sales: {category_lowest_sales}")
print(f"Total sales for this category: {sales_per_category[category_lowest_sales]}")

Low impact category based on quantity: Furniture
Total quantity for this category: 14

Low impact category based on sales: Appliances
Total sales for this category: $ 5,878.30


#### 5. Identify the top selling products

In [13]:
# Calculate the unique count of items in a product ID
unique_count = s_df['ProductID'].nunique()


# Validate the top N selling input by the user
while True:
    try:
        top_n_products = int(input("Please enter your desired top N selling products:"))
        if top_n_products <= 0 or top_n_products > unique_count:
            print(f"Please enter a number between 1 and {unique_count}")
        else:
            break
    except ValueError:
        print("Invalid input. Please enter a valid positive integer.")
print()


# Group quantity and sales data by category and Region and calculate the total quantity and total sales for each category
quantity_sales_per_category = s_df.groupby(["Category", "Region"])[["Quantity", "Sales($)"]].sum()

# Group quantity and sales data by category and product ID and calculate the total quantity and total sales for each product ID
quantity_sales_per_product = s_df.groupby(["ProductID", "Category"])[["Quantity", "Sales($)"]].sum()


# Sort total quantity per product in descending order and select top N products
top_selling_products_quantity = quantity_sales_per_product.sort_values(by = "Quantity", ascending = False).head(top_n_products)

# Format sales totals with commas and keep them rounded to 2 decimal places only for the "Sales($)" column
top_selling_products_quantity_formatted = top_selling_products_quantity.copy()
top_selling_products_quantity_formatted["Sales($)"] = top_selling_products_quantity_formatted["Sales($)"].map(lambda x: "$ {:,.2f}".format(x))

# Concatenate the "Quantity" column with the formatted "Sales($)" column
top_selling_products_quantity_formatted["Quantity"] = top_selling_products_quantity["Quantity"]


# Sort total sales per product in descending order and select top N products
top_selling_products_sales = quantity_sales_per_product.sort_values(by = "Sales($)", ascending = False).head(top_n_products)

# Format sales totals with commas and keep them rounded to 2 decimal places only for the "Sales($)" column
top_selling_products_sales_formatted = top_selling_products_sales.copy()
top_selling_products_sales_formatted["Sales($)"] = top_selling_products_sales_formatted["Sales($)"].map(lambda x: "$ {:,.2f}".format(x))

# Concatenate the "Quantity" column with the formatted "Sales($)" column
top_selling_products_sales_formatted["Quantity"] = top_selling_products_sales["Quantity"]


# Sort total sales per product in descending order
top_selling_products_category = quantity_sales_per_category.sort_values(by = "Sales($)", ascending = False)

# Format sales totals with commas and keep them rounded to 2 decimal places only for the "Sales($)" column
top_selling_products_category_formatted = top_selling_products_category.copy()
top_selling_products_category_formatted["Sales($)"] = top_selling_products_category_formatted["Sales($)"].map(lambda x: "$ {:,.2f}".format(x))

# Concatenate the "Quantity" column with the formatted "Sales($)" column
top_selling_products_category_formatted["Quantity"] = top_selling_products_category["Quantity"]


# Print selected columns for top selling products
print(f"The top {top_n_products} selling products based on quantity are:")
print()
print(top_selling_products_quantity_formatted)
print()
print(f"The top {top_n_products} selling products based on sales are:")
print()
print(top_selling_products_sales_formatted)
print()
# Print selected columns for product category ranking based on sales
print("The product category ranking based on sales is:")
print()
print(top_selling_products_category_formatted)

Please enter your desired top N selling products: 5



The top 5 selling products based on quantity are:

                       Quantity    Sales($)
ProductID Category                         
A6789     Appliances          6  $ 2,413.50
A1234     Apparel             4  $ 1,381.50
E2345     Electronics         4  $ 2,621.40
F5678     Furniture           4  $ 3,642.20
E8903     Electronics         2  $ 1,360.60

The top 5 selling products based on sales are:

                       Quantity    Sales($)
ProductID Category                         
F5678     Furniture           4  $ 3,642.20
E2345     Electronics         4  $ 2,621.40
A6789     Appliances          6  $ 2,413.50
F1236     Furniture           2  $ 1,841.80
F3457     Furniture           2  $ 1,740.40

The product category ranking based on sales is:

                    Quantity     Sales($)
Category    Region                       
Furniture   North         14  $ 12,767.00
Electronics South         14   $ 9,385.00
Apparel     West          17   $ 6,048.25
Appliances  East       

#### 6. Based on your analysis what recommendation would you give to improve sales in November?

Based on the analysis, it appears that the sales in November could be improved by offering promotions on top-selling products that are in higher demand and/or generate the most revenue. Products recommended for promos include A6789 (Appliance), E2345 (Electronics), and F5678 (Furniture). 

Concerning marketing strategies, furniture in the North should be prioritised to maximize profit since it has the highest revenue potential. Whereas Apparel in the West should be prioritised to expand market share or reach a broader customer base since it appears to resonate most with customers and thus can drive the volume of sales. Regionally for Electronics, strategies should be tailored to the specific needs and preferences of customers in the East region to improve sales and thus drive revenue growth and market share expansion.

In addition, expanding the product range to offer products at different price points or with varying features is recommended for appliances in the East which has a low impact primarily due to low revenue generation despite reasonable sales volume. This is to capture customers willing to pay a premium for additional functionality. 

Furthermore, the product range can be expanded to cater to diverse customer preferences for furniture in the North which has a low impact primarily due to low sales volume or market penetration despite generating the most revenue. This could involve variations in size, color, design, or functionality to appeal to diverse customer segments. 

In conclusion, focusing on marketing strategies, offering promotions on top-selling products, and expanding the product range to cater to diverse customer preferences based on customer feedback unique to each region is highly recommended.

### Sales DataFrame at the End of the Program

In [14]:
# Display dataframe at the end of the program
s_df

,Date,Region,Category,Sales($),ProductID,Month,Quantity
0,2023-09-01,East,Electronics,1200.50,E1234,9,1
1,2023-09-02,West,Apparel,350.75,A4567,9,1
2,2023-09-03,North,Furniture,890.20,F7890,9,1
3,2023-09-04,South,Electronics,550.40,E2345,9,1
4,2023-09-05,East,Appliances,420.60,A6789,9,1
...,...,...,...,...,...,...,...
56,2023-10-27,North,Furniture,870.20,F3457,10,1
57,2023-10-28,South,Electronics,730.40,E4568,10,1
58,2023-10-29,East,Appliances,410.60,A6789,10,1
59,2023-10-30,West,Apparel,320.25,A7891,10,1
